# Spark GraphFrames - Student Version

Before running present notebook please install the GraphFrames package into your cluster by following the steps in the pdf available with this notebook.  
To check you have everything setup correctly run the below code and make sure there are no errors. If you get a message about connecting to a cluster click ok.

In [0]:
#import the package we just installed
from graphframes import *
#import data types - All data types of Spark SQL are located in the package of pyspark.sql.types
from pyspark.sql.types import *
#row can be used to create a row object by using named arguments
from pyspark.sql import Row
from pyspark.sql.functions import col


<b>Graphs:</b> Graphs are mathematical structures used to model pairwise relations between objects. A graph in this context is made up of vertices (also called nodes or points) which are connected by edges (also called links or lines). A distinction is made between undirected graphs, where edges link two vertices symmetrically, and directed graphs, where edges link two vertices asymmetrically; see Graph (discrete mathematics) for more detailed definitions and for other variations in the types of graph that are commonly considered (follow this link: https://en.wikipedia.org/wiki/Graph_(discrete_mathematics)). Graphs are one of the prime objects of study in discrete mathematics.

![image](https://mathinsight.org/media/image/image/small_undirected_network_labeled.png)

Graph processing is important aspect of analysis that applies to a lot of use cases. Fundamentally graph theory and processing are about defining relationships between different nodes and edges. Nodes or vertices are the units while edges define the relationships between nodes. This works great for social network analysis and running algorithms like PageRank to better understand and weigh relationships.

Some business use cases could be to look at central people in social networks [who is most popular in a group of friends], importance of papers in bibliographic networks [which papers are most referenced], and of course ranking web pages.

## Pagerank

For this class we will look into the page rank algorithm again but this time  from another point of view. As you might remenber, the page rank of a particular web page indicates its relative importance within a group of web pages. The higher the page rank, the higher up it will appear in a search result list. The importance of a page is defined by the importance of all the web pages that provide an outbound link to the web page in consideration. For example, say that web page X has very high relative importance. Web page X is outbounding to web page Y (webpage X links to Y); hence, web page Y will also have high importance. However, if webpage X links to many webpages, including Y, the importance of Y is lower.

For more information on pagerank you either:

Follow through the worked example: https://www.cs.princeton.edu/~chazelle/courses/BIB/pagerank.htm

Take a look at one of the many youtube explanations, for example (10min): https://www.youtube.com/watch?v=qpIqLjfcabs

Part of what is important here is to understand the data format we use to represent a graph. I have commented the steps below:

In [0]:
# First we start with the list of vertices, I have 5 vertices here, each named with a letter of the alphabet
verticesList = ['a', 'b', 'c', 'd']
# We load this list into an RDD as we have seen before
verticesListRDD = sc.parallelize(verticesList, 1)
# We convert this RDD to a dataframe using row objects and specifying a schema.
verticesListRowsRDD = verticesListRDD.map( lambda data : Row(data))
verticesSchema = StructType([StructField('id', StringType(), True)])
verticesDataFrame = verticesListRowsRDD.toDF(verticesSchema).persist()
#Here I show the schema of the Dataframe.
verticesDataFrame.show()

Next we need to know how the vertices are connected, a list of edges. The process is very similar to that we just saw.

First, we have to create a list of edges. Each edge in the list will be defined by a tuple.

Then, we have to create an RDD of edges.

Thereafter, we have to transform our RDD to an RDD of row objects. This will be followed by creating a schema and a DataFrame of edges. Let’s perform the steps:

In [0]:
edgeDataList = [('a','b'), ('a','c'), ('a','d'), ('b', 'c'),
('b', 'd'),('c', 'b'), ('d', 'a'), ('d', 'c')]
sourceColumn = StructField('src', StringType(),True)
destinationColumn = StructField('dst', StringType(), True)
edgeSchema = StructType([sourceColumn, destinationColumn])
edgeRDD = sc.parallelize(edgeDataList, 1)
edgeRDDRows = edgeRDD.map( lambda data : Row(data[0], data[1]))
edgeDataFrame = edgeRDDRows.toDF(edgeSchema).persist()
edgeDataFrame.show()

At this moment, we have `verticesDataFrame`, a DataFrame of vertices; and `edgeDataFrame`, a DataFrames of edges. Using these two, we can create our graph. The GraphFrame Python class is defined under the `graphframes.graphframe` submodule. GraphFrame() takes the vertices and edges DataFrames and returns a GraphFrames object. We have our `GraphFrames` object, `ourGraph`. We can fetch all the vertices as follows: In `GraphFrames`, we can create a graph by using the following code lines:

**If you get an error on the below cell make sure you installed the correct version of dataframes**

In [0]:
ourGraph = GraphFrame(verticesDataFrame, edgeDataFrame)
ourGraph.vertices.show()
ourGraph.edges.show()

#### VISUALIZATION OF THE NETWORK
note that the library that i will use to visualize the network is not a big data scale graph visualization tool. This serves ONLY for you to have a idea of the graph we just created.

for you reference big data capable software tools are:

* Gephi: Gephi is a popular open-source network visualization tool capable of handling networks with up to a million nodes and edges.

* Graphistry: Graphistry uses GPU acceleration for handling and visualizing large graphs and can be integrated with PySpark. However, it's a commercial product.


In [0]:
# Requirements
!pip install 'scipy>=1.8'
!pip install 'networkx<2.7'

In [0]:
import networkx as nx
import matplotlib.pyplot as plt

# Create a new directed graph
G = nx.DiGraph()

# Add vertices to the graph
for vertex in verticesList:
    G.add_node(vertex)

# Add edges to the graph
for edge in edgeDataList:
    G.add_edge(edge[0], edge[1])

# Draw the graph
nx.draw(G, with_labels=True, node_color='lightblue', font_weight='bold', arrows=True)
plt.show()

### Some simple analysis

In [0]:
# View the inDegrees (the number of edges directed into a vertex)


In [0]:
# View the outDegrees (the number of edges directed out of a vertex)


Page rank for pages can be found by using the `pageRank()` function, which is defined on the GraphFrames object.

`pageRank()` function take two arguments: first is resetProbability (is the alpha explained below) and the second is maxIter

Informally Page Rank Explanation: consider someone browsing the web ("surfer"). At every page, the surfer either follows a link on the page to another page or does something else. When following a link, without any other information, the surfer picks a link from the page at random and follows that one. When “doing something else,” the surfer moves to a random page on the web and restarts the surfing. To generate a simple model, we assume both of these behaviors even though they may seem ridiculous. When stated mathematically, this random surfer model is called a Markov chain because the behavior of the surfer only depends upon the current page and not the history of previous pages. Let α be the probability that the surfer follows a link; then 1 − α is the probability that the surfer “does something else.” 

Now imagine that we let the surfer run for a long time. The PageRank of a page is the probability of finding the surfer at that page as the surfing time becomes infinite. A key assumption behind PageRank is that pages where we are more likely to find the random surfer are more important pages and thus we can view the PageRank as a measure of the page’s importance.

<img src="https://upload.wikimedia.org/wikipedia/commons/6/69/PageRank-hi-res.png" style="width:750px">

*Source: https://web.stanford.edu/group/SOL/dissertations/pagerank-sensitivity-thesis-online.pdf*

In [0]:
# Calculate the Pagerank with resetProbability=0.15 and maxIter = 5


## More graphframes applications

Let's look at a slightly more interesting dataset. Look at the code below, you should see we are loading a number of people as vertices, then they can either connect as friends (bi-directional) or one can follow another (uni directional).

In [0]:

vertices = spark.createDataFrame([('1', 'Carter', 'Derrick', 50), 
                                  ('2', 'May', 'Derrick', 26),
                                 ('3', 'Mills', 'Jeff', 80),
                                  ('4', 'Hood', 'Robert', 65),
                                  ('5', 'Banks', 'Mike', 93),
                                 ('98', 'Berg', 'Tim', 28),
                                 ('99', 'Page', 'Allan', 16)],
                                 ['id', 'name', 'firstname', 'age'])
edges = spark.createDataFrame([('1', '2', 'friend'), 
                               ('2', '1', 'friend'),
                              ('3', '1', 'friend'),
                              ('1', '3', 'friend'),
                               ('2', '3', 'follows'),
                               ('1', '4', 'follows'),
                               ('4', '5', 'follows'),
                               ('3', '4', 'friend'),
                               ('4', '3', 'friend'),
                               ('5', '3', 'friend'),
                               ('3', '5', 'friend'),
                               ('4', '5', 'follows'),
                              ('98', '99', 'friend'),
                              ('99', '98', 'friend')],
                              ['src', 'dst', 'type'])
g = GraphFrame(vertices, edges)
## Take a look at the DataFrames
g.vertices.show()
g.edges.show()
## Check the number of edges of each vertex
g.degrees.show()

Undirected graphs have edges that do not have a direction. The edges indicate a two-way relationship, in that each edge can be traversed in both directions. If your DataFrame only consist of two-way directed edges, you may be interested in analyzing undirected edges. You can convert your graph by mapping a function over the edges DataFrame that deletes the row if src ≥ dst (or the other way around). Then each edge is only included once.

Start from the dataframe called edges, notice that directional edges are listed [a,b, follows] where a < b (so the lowest number edge is listed first), uniderctional edges are listed twice.



In [0]:
# Create a new dataframe edge that only contains edges in one direction (src > dst) to identify the friends links


We can filter edges and vertices directly as well. If we want to work with a GraphFrame with only follows and people over 30 for example we can use a subgraph, for example:

In [0]:
# filterEdges with "type = 'follows'" and filterVertices with "age > 30" and show the edges


<b> Degrees </b> One question we might be interested in is how many inbound and outbound connections each vertices has, we can use the inDegrees and outDegrees functions to calculate this:

<b> Motif </b> All networks, including biological networks, social networks, technological networks (e.g., computer networks and electrical circuits) and more, can be represented as graphs, which include a wide variety of subgraphs. One important local property of networks are so-called network motifs, which are defined as recurrent and statistically significant sub-graphs or patterns.

Network motifs are sub-graphs that repeat themselves in a specific network or even among various networks. Each of these sub-graphs, defined by a particular pattern of interactions between vertices, may reflect a framework in which particular functions are achieved efficiently. Indeed, motifs are of notable importance largely because they may reflect functional properties. They have recently gathered much attention as a useful concept to uncover structural design principles of complex networks. Although network motifs may provide a deep insight into the network’s functional abilities, their detection is computationally challenging.

GraphFrame motif finding uses a simple Domain-Specific Language (DSL) for expressing structural queries. For example, graph.find("(a)-[e]->(b); (b)-[e2]->(a)") will search for pairs of vertices a,b connected by edges in both directions. It will return a DataFrame of all such structures in the graph, with columns for each of the named elements (vertices or edges) in the motif. In this case, the returned columns will be “a, b, e, e2.”

DSL for expressing structural patterns:

The basic unit of a pattern is an edge. For example, "(a)-[e]->(b)" expresses an edge e from vertex a to vertex b. Note that vertices are denoted by parentheses (a), while edges are denoted by square brackets [e].

A pattern is expressed as a union of edges. Edge patterns can be joined with semicolons. Motif "(a)-[e]->(b); (b)-[e2]->(c)" specifies two edges from a to b to c.

Within a pattern, names can be assigned to vertices and edges. For example, "(a)-[e]->(b)" has three named elements: vertices a,b and edge e. These names serve two purposes:

*  The names can identify common elements among edges. For example, "(a)-[e]->(b); (b)-[e2]->(c)" specifies that the same vertex b is the destination of edge e and source of edge e2.

*  The names are used as column names in the result DataFrame. If a motif contains named vertex a, then the result DataFrame will contain a column “a” which is a StructType with sub-fields equivalent to the schema (columns) of GraphFrame.vertices. Similarly, an edge e in a motif will produce a column “e” in the result DataFrame with sub-fields equivalent to the schema (columns) of GraphFrame.edges.

*  Be aware that names do not identify distinct elements: two elements with different names may refer to the same graph element. For example, in the motif "(a)-[e]->(b); (b)-[e2]->(c)", the names a and c could refer to the same vertex. To restrict named elements to be distinct vertices or edges, use post-hoc filters such as resultDataframe.filter("a.id != c.id").

It is acceptable to omit names for vertices or edges in motifs when not needed. E.g., "(a)-[]->(b)" expresses an edge between vertices a,b but does not assign a name to the edge. There will be no column for the anonymous edge in the result DataFrame. Similarly, "(a)-[e]->()" indicates an out-edge of vertex a but does not name the destination vertex. These are called anonymous vertices and edges.

An edge can be negated to indicate that the edge should not be present in the graph. E.g., "(a)-[]->(b); !(b)-[]->(a)" finds edges from a to b for which there is no edge from b to a.

For more information: http://graphframes.github.io/graphframes/docs/_site/user-guide.html#motif-finding

A worked example is available here with a few motif patterns applied. They use Scala as opposed to Python but you can ignore the code outside of the patterns they use: http://www.twesdai.com/2017/09/15/motif-analysis-using-apache-spark-graphframes/

Let's look at a simple example to try to make how this works clear. 

In [0]:
# find where a is connected to b, but b is not connected to a


We could find the same by specifing a condition on the edges by naming the edge in the find e1 and using filter "e1.type = 'follows'"

Find everyone connected to someone under 30. Display their id, name and age using select on "b.id","b.name","b.age"

#### Pratice Exercises

Find all friend relationships without filtering on edges (so specifying the network structure of the motif you want to find).

Find all the people who are older than 60 and have at least one friend.

<b>Connected components</b> A connected component of a graph is a subgraph in which any two vertices are connected to each other by one or more edges, and which is connected to no additional vertices in the supergraph. In the graph below there would be 5 connected component subgraphs:

![image](http://lemon.cs.elte.hu/pub/doc/1.2.3/connected_components.png)



Graphframes can compute the connected component membership of each vertex and return a DataFrame with each vertex assigned a component ID. The GraphFrames connected components implementation can take advantage of checkpointing to improve performance. This will take up tp 10 min to run.

In [0]:
sc.setCheckpointDir("/tmp/graphframes-example-connected-components")
# find and show connectedComponents 


In [0]:
# find and show stronglyConnectedComponents with maxIter=10  


The Label Propagation algorithm (LPA) is a fast algorithm for finding communities in a graph. In the context of social networks, a community could represent users with shared interests or characteristics. Apply the label propogation algorithm with a max of 100 iterations.

In [0]:
# find communities with labelPropagation using maxIter=10

# Display the detected communities by id and label


Let's use this information to colour our graph.

In [0]:
import numpy as np
# Create a directed graph
G = nx.DiGraph()

#convert results to pandas
pdf = result_scc.orderBy("id").toPandas()

# Add edges
for row in g.vertices.collect():
    G.add_node(row['id'])

# Add edges
for row in g.edges.collect():
    G.add_edge(row['src'], row['dst'])

# Create a color map based on the component id
color_map = {}

# Get all unique components and create a color for each of them
unique_components = pdf[pdf.columns[-1]].unique()
colors = plt.cm.rainbow(np.linspace(0, 1, len(unique_components)))

for component, color in zip(unique_components, colors):
    color_map[component] = color

# Map the nodes to their colors
node_colors = [color_map[component] for component in pdf[pdf.columns[-1]]]

# Draw the graph
nx.draw(G, node_color=node_colors, with_labels=True)
plt.show()


####Triangle Count:
The triangle count for a node in a graph is a measure of the cohesiveness of the user's immediate network, which can help identify tightly-knit communities.

The Triangle Count algorithm is a graph analysis algorithm that counts the number of triangles in a graph. A triangle in a graph consists of three nodes that are connected to each other, forming a closed loop. The Triangle Count algorithm helps identify the presence of triangles and provides a measure of the cohesiveness or clustering of nodes within a graph.

In [0]:
# Run triangleCount algorithm

# Display the number of triangles for each user


What does the result say about how are people connected in our graph? Take a look at who is connected to who in the data and see if you can match a pattern to what is being reported (the component number is not important, only which nodes have the same component).

<b> Shortest path </b> Graphframes includes a simple shortest path algorithm, it finds the distance between all nodes and a given set of landmarks.

In the example below we will see from each node how many steps does it take to get to a landmark. So for example [5 -> 1, 1 -> 1] would means that to get to vertice named 5 takes 1 edge, to get to a vertice named 1 also takes 1 step.

In [0]:
# Find shortestPaths between landmarks ["1", "5"] 


<b>Search</b> Breadth-first search (BFS) is an algorithm for traversing or searching tree or graph data structures. It starts at the tree root (or some arbitrary node of a graph, sometimes referred to as a 'search key'[1]), and explores all of the neighbor nodes at the present depth prior to moving on to the nodes at the next depth level.

It uses the opposite strategy as depth-first search, which instead explores the highest-depth nodes first before being forced to backtrack and expand shallower nodes.

For more information on these alogrithms: https://en.wikipedia.org/wiki/Breadth-first_search

Let's apply bfs using the fromExpr and toExpr arguments to find a path from id 1 to id 5.

In [0]:
# use bfs with fromExpr="id='1'", toExpr = "id='5'" 


we can be even more specific for example find a path from id 1 to anyone over 65, but not using any friend edges (edgeFilter argument) and setting a max length of 4 (maxPathLength argument). It can also be useufl to drop duplicates.

In [0]:
# use bfs with fromExpr="id='1'",  toExpr = "age >65", edgeFilter = "type != 'friend'", maxPathLength = 4


## Exercise

In the following exercises you will use a data extract from instagram that represents a soical network. You will need to upload the data to the dbfs using the upload command under file.

Then the below commands will load the data into dataframes to work with.

In [0]:
nodes_df = spark.read.format("csv").option("header", "true").load("dbfs:/FileStore/shared_uploads/iscott@novaims.unl.pt/instagram_nodes.csv")
edges_df = spark.read.format("csv").option("header", "true").load("dbfs:/FileStore/shared_uploads/iscott@novaims.unl.pt/instagram_edges___directed.csv")

Let's take a look at the data:

In [0]:
display(nodes_df)

In [0]:
display(edges_df)

In [0]:
# create a graphframes graph from the data:


How many edges and vertices are their in our graph?

In our dataset who is followed the most (including friends) and who follows the most in our dataset (hint: think degrees)

Apply label propogation algorithm and find how many unique labels exist. Bonus: use Network X to plot a coloured graph

Who is the most influential by pagerank and by number of triangles?

For each node find all follower of followers who are not directly followed.

For each node find all mutual followers of mutual followers who are not directly followed or mutually followed.

What is the shortest distance from our top influencer (by pagerank) to someone older than them?

## Bonus Exercise: Bikeshare

Here is your chance to apply what you have learned to some real data. We will be using a bike sharing dataset, but a very different one from last class

Here you’re going to be working with publicly available bike data from the Bay Area Bike Share portal, specifically analyzing the 2017 year of data. First we will download the data and read it into spark, we will also rename some of the columns to meet the requirements of graphframes.

In [0]:
%sh wget https://raw.githubusercontent.com/udacity/data-analyst/master/projects/bike_sharing/201508_trip_data.csv

In [0]:
%sh wget https://raw.githubusercontent.com/udacity/data-analyst/master/projects/bike_sharing/201508_station_data.csv

In [0]:
tripDF = spark.read.csv('file:/databricks/driver/201508_trip_data.csv',header=True, inferSchema = True)
tripDF.printSchema()

In [0]:
stationDF = spark.read.csv('file:/databricks/driver/201508_station_data.csv',header=True, inferSchema = True)
stationDF.printSchema()

In [0]:
stationVertices = stationDF.withColumnRenamed("name", "id").distinct()
tripEdges = tripDF.withColumnRenamed("Start Station", "src").withColumnRenamed("End Station", "dst")

In [0]:
tripEdges.show(10, False)

In [0]:
stationVertices.show(10, False)

Alright you now have the data formatted as you need to perform analysis. Take a look to make sure you understand what the data is. Basically you have bike stations (verticies) and trips between the stations (edges). 

Hint: in the cell above I used 'False' in the `show()`, this will be useful for you to prevent truncating station names. The highest ranked station should be San Jose Diridon Caltrain Station.

#####Create a graphframe and perform a pagerank to find the most import stations (make sure to set the maxIterations to 5).

In [0]:
#take the vertices and edges DataFrames and returns a GraphFrames object.


In [0]:
#show the edges


In [0]:
#Run the pageRank()


In [0]:
#Show() the vertices, ranked


One question is: what are the most common trip paths? You can do this by performing a grouping operator and adding the edge counts together, think of how we used sparkSQL to implement SQL like queries last week. Remember the list of edges is just a dataframe so you can use SQL queries to find a result like this. In case you have forgotten this link has some information on groupBy:

https://sparkbyexamples.com/spark/using-groupby-on-dataframe/

The most common trip should be:

San Francisco Caltrain 2 (330 Townsend)  ->    Townsend at 7th.

Create a new graphframe with only unique edges (you can use the dataframe you created above) and check if all stations are connected.

In [0]:
#Create a gbsUnique graphframe


In [0]:
#Computes the strongly connected component membership of each vertex and returns a graph with each vertex assigned a component ID.



In [0]:
#show the graphframe with component


Remember that in this instance you’ve got a directed graph. That means that your trips are directional - from one location to another. Therefore you get access to a wealth of analysis that you can use. You can find the number of trips that go into a specific station and leave from a specific station.

One interesting question you could ask is what is the station with the highest ratio of in degrees to out degrees. As in, what station acts as a pure trip sink. A station where trips end at but rarely start from. This station will end up with a lot of excess bikes that will need to be re-distributed.

Consider the station with the highest ratio of intrips to outtrips, find all stations connected by 1 or 2 trips to this station using motifs.

In [0]:
#store the intermediate computation so they can be reused in subsequent actions.


In [0]:
#count the number os connections


In [0]:
# Find stations connected by one trip
